# GVA Data Access Lab

This lab is to help build a familiarity with accessing data on the GVA platform.

A Python library is maintained to assist with working with GVA, it is not available on PyPI but can pip installed using the following command:

In [ ]:
!pip install --upgrade git+https://github.com/gva-jjoyce/gva_data

We are primarily going to use two components in the GVA library; the `Reader` and `dictset`. 

The `Reader` component helps with reading data, taking care of activities like uncompressing compressed data, joining data across partitions and searching across multiple days. `dictset` helps processing data.

Once read, data can be loaded into `Pandas` to perform analysis. Whilst this is valid approach for most datasets, Pandas is memory intensive so some data may never be able to be loaded into Pandas or may need some treatment before loading, this is the purpose of the `dictset` component.

Additional information on these libraries is available in the GitHub repo:

https://github.com/gva-jjoyce/gva_data/blob/main/docs/gva.data.readers.md

https://github.com/gva-jjoyce/gva_data/blob/main/docs/gva.data.formats.dictset.md

In [9]:
from gva.data.readers import Reader, MinioReader
from gva.data.formats import dictset
from IPython.display import HTML, display

The data in part one of this lab is from [FiveThirtyEight](https://fivethirtyeight.com/), the source data is available on the [FiveThirtyEight's GitHub account](https://github.com/fivethirtyeight/data/tree/master/star-wars-survey) and was the data used in Walt Hickey's [America’s Favorite ‘Star Wars’ Movies](https://fivethirtyeight.com/features/americas-favorite-star-wars-movies-and-least-favorite-characters/) article.

To read the data, we use the `Reader` class and give it the path of data, the assigned variable is a _generator_, which behaves a lot like a _list_, but doesn't load the entire dataset into memory all at once. If we want to cycle over the set more than once we need to create a new `Reader` instance each time or convert the _generator_ to a _list_. Because the dataset is quite small, we will just covert the `Reader` to a _list_.

In [10]:
star_wars_survey_results = list(Reader(
        reader=MinioReader,
        from_path='SNAPSHOTS/DEMO/538/StarWars/star_wars.jsonl',
        end_point='10.10.10.30:9000',
        access_key='57BTIM68ETSQ7ZQG',
        secret_key='LXWODW6DSZX9AD9TX9XBTW292KEOATGB',
        secure=False))

Most data on GVA has a README file which includes details of the schema of the file, once we have the data loaded we can display it similar to _Pandas_ using the `dictset.to_html_table` method (there is also a `dictset.to_ascii_table` method). This method with exhaust _generators_ (a new generator will need to be created to use the data) and is intended for exploration only. We've converted our dataset to a _list_ to avoid this issue.

In [8]:
display(HTML(dictset.to_html_table(star_wars_survey_results)))

We can see the responses to the first question are 'Yes'/'No', we're going to use `dictset.set_column` to convert these strings to a boolean. `dictset.set_column` takes three paramters:

- The dictset to act on
- The column to update or create
- A setter, either a fixed value or a Callable to calculate the column

Although a _lambda_ could be used as the _setter_, we're going to define a function to use as the setter as this is generally more reusable and easier to read and therefore debug.

This function takes a column name and returns a function, this allows us to provide the name of the field rather than hard-coding the column name and could be used to convert any of ther other 'Yes'/'No' columns.

We're also going to convert the resultant dataset to a list so we can iterate over it a number of times.

In [ ]:
def yes_no_to_boolean(column):
    # return a function to be called for each row
    def process_row(row):
        return str(row.get(column)).lower() == 'yes'
    return process_row

yes_no_converted = list(dictset.set_column(
        star_wars_survey_results, 
        'Have you seen any of the 6 films in the Star Wars franchise?',
        setter=yes_no_to_boolean('Have you seen any of the 6 films in the Star Wars franchise?')))

display(HTML(to_html_table(yes_no_converted)))

## CHALLENGE ONE

Your first challenge is to create a new field 'Greedo Shot First' based on the 'Which character shot first?' column. This new column should be set to `True` for every row where the respondent answered 'Greedo' for this column.

You should call the resulting dataset _greedo_shot_first_, if you have the right answer the _TEST_ statement a few cells down will show success.

In [ ]:
# CHALLENGE ONE TEST

if len(list(dictset.select_from(greedo_shot_first, where=lambda row: row['Greedo Shot First']))) == 197:
    display(HTML("<img src='https://media.giphy.com/media/111ebonMs90YLu/source.gif' width='480' align='center'>"))
else:
    display(HTML("<img src='https://media.giphy.com/media/3ohuPwtVfPsxaMp0QM/giphy.gif' width='480' align='center'>"))

Selecting and filtering data is a common activity, simple actions can be done using `dictset`, either before or instead of loading into _Pandas_.

We're going to count the people who responded that they liked Anakin but that they didn't like Vader. To do this we'll use `dictset.select_from`. This method takes three parameters:

- The dictset to act on
- columns - the list of columns to select (optional, default is all columns)
- where - a function to filter rows (optional, default is include all records)

There _where_ parameter can be a _lambda_, but again we'll define a function.

In [ ]:
def likes_anakin_but_not_vader(row):
    # These are what we're classing as a positive or a negative response
    positive_responses = ['Somewhat favorably', 'Very favorably']
    negative_responses = ['Somewhat unfavorably', 'Very unfavorably']
    
    # Return True where Anakin is positive and Vader is negative
    # rows that evaluate to True are kept, False are removed
    return row['Anakin Skywalker'] in positive_responses and row['Darth Vader'] in negative_responses

# Execute the selection against the star_wars data, using the function we defined above
who_likes_anakin_but_not_vader = list(dictset.select_from(
                star_wars_survey_results,
                where=likes_anakin_but_not_vader))

# Count the number 
f"{len(who_likes_anakin_but_not_vader)} people reponded favorably about Anakin but unfavorably about Vader"

## CHALLENGE TWO

The numbers associated with each movie in the dataset is the order the respondent liked each movie, with 1 being their most favorite and 6 their least favorite.

Your next challenge, and last with the Star Wars dataset, is working out how many people liked 'The Phantom Menace' more than 'The Empire Strikes Back'. You should call your dataset _likes_jarjar_over_yoda_, if you have the right answer the _TEST_ statement a few cells down will show success.

If you are having trouble, be aware of the spaces in the movie titles.

In [ ]:
# CHALLENGE TWO TEST

if len(list(likes_jarjar_over_yoda)) == 214:
    display(HTML("<img src='https://media.giphy.com/media/oGO1MPNUVbbk4/giphy.gif' width='480' align='center'>"))
else:
    display(HTML("<img src='https://media.giphy.com/media/3ohzdMibqeBjRPX53W/giphy.gif' width='480' align='center'>"))